# Reports PDF Export - Local Test Notebook

This notebook helps you test ReportLab PDF generation for reports without deploying.

- Requires `reportlab` installed in your environment.
- Generates a sample Overview PDF using `ReportsPDFExporter`.
- Saves output to a local `exports/` folder and shows a quick preview/link.

If you need other sections (performance/diversion), we can extend this notebook later.


In [ ]:
# Setup import path and check dependencies
import os, sys

# Ensure project root is on sys.path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
	sys.path.insert(0, project_root)

# Dependency check
try:
	import reportlab
	print("reportlab version:", getattr(reportlab, '__version__', 'unknown'))
except Exception as e:
	print("ReportLab not available - install it with: pip install reportlab")
	raise

# Import the PDF exporter
from GEPPPlatform.services.cores.reports.pdf_export import ReportsPDFExporter
print("Loaded ReportsPDFExporter successfully")


In [ ]:
# Generate a sample Overview PDF
from datetime import datetime, timezone

# Minimal mock of the Overview payload expected by the exporter
overview_mock = {
	'key_indicators': {
		'total_waste': 12345.678,
		'recycle_rate': 42.5,
		'ghg_reduction': 987.65,
	},
	'waste_type_proportions': [
		{'category_id': 1, 'category_name': 'Recyclable', 'total_waste': 5000, 'proportion_percent': 40.5},
		{'category_id': 3, 'category_name': 'Organic', 'total_waste': 3000, 'proportion_percent': 24.3},
		{'category_id': 4, 'category_name': 'General', 'total_waste': 2500, 'proportion_percent': 20.2},
		{'category_id': 2, 'category_name': 'Hazardous', 'total_waste': 1845.678, 'proportion_percent': 14.9},
	]
}

# Date range for display
now_utc = datetime.now(timezone.utc)
date_from = (now_utc.replace(month=1, day=1, hour=0, minute=0, second=0, microsecond=0)).isoformat()
date_to = now_utc.isoformat()

# Generate PDF response
pdf_response = ReportsPDFExporter.generate_overview_pdf(
	overview=overview_mock,
	date_from=date_from,
	date_to=date_to,
	timezone_name='Asia/Bangkok'
)

# Write to local file
import base64
output_dir = os.path.join(os.getcwd(), 'exports')
os.makedirs(output_dir, exist_ok=True)
file_path = os.path.join(output_dir, pdf_response['filename'])
with open(file_path, 'wb') as f:
	f.write(base64.b64decode(pdf_response['body']))

print('PDF written to:', file_path)


In [ ]:
# Preview (link and inline if supported)
from IPython.display import FileLink, IFrame

print('Click to download:')
FileLink(file_path)

try:
	display(IFrame(src=file_path, width=900, height=600))
except Exception:
	print('Inline preview not supported in this environment. Use the link above.')
